In [45]:
import numpy as np
import random
from pprint import pprint
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [38]:
sensor_neighbors = {}
for i, (key, val) in enumerate(SENSORS.items()):
    distances = {}
    for j, (key2, val2) in enumerate(SENSORS.items()):
        if key != key2:
            distance = haversine_np(val[0], val[1], val2[0], val2[1])
            distances[key2] = distance
    sorted_distances = sorted(distances.items(), key=lambda kv: kv[1])
    sensor_neighbors[key] = sorted_distances[0:5]

In [39]:
sensor_tower_neighbors = {}
for i, (key, val) in enumerate(SENSORS.items()):
    distances = {}
    for j, (key2, val2) in enumerate(TOWERS.items()):
        distance = haversine_np(val[0], val[1], val2[0], val2[1])
        distances[key2] = distance
    sorted_distances = sorted(distances.items(), key=lambda kv: kv[1])
    sensor_tower_neighbors[key] = sorted_distances[0:5]

In [40]:
def inspect_data_structures(current_sensor, time, to_edge, data_structure):
    print("Current sensor: ", current_sensor)
    print("Time: ", time)
    print("******** SP ********")
    pprint(sensor_path)
    print("******** EL ********")
    pprint(edge_list)

In [56]:
def update_edge_list(current_sensor, time, to_edge, edge_list):
    """
    update in memory graph data structure
    Structure:
        {current_sensor: {time: next0}, 
         next0: {1: next1},
         next1: {2: next2}
    """
    # current_sensor --- time ---> to_edge
    if current_sensor in edge_list.keys():
        if time in edge_list[current_sensor].keys():
            print("CONSTRUCTION ERROR: {} already has an edge at time {}".format(current_sensor, time))
            inspect_data_structures(current_sensor, time, to_edge, edge_list)
            raise ValueError
        else:
            edge_list[current_sensor][time] = to_edge
    else:
        edge_list[current_sensor] = {time: to_edge}
    return edge_list

def update_sensor_path(starting_sensor, time, to_edge, sensor_path):
    """
    update in memory graph data structure
    # {starting_sensor : {time:to_edge}}
    """
    # current_sensor --- t ---> to_edge
    if starting_sensor in sensor_path.keys():
        if time in sensor_path[starting_sensor].keys():
            print("CONSTRUCTION ERROR: {} already has an edge at time {} in sensor_path".format(starting_sensor, time))
            inspect_data_structures(starting_sensor, time, to_edge, sensor_path)
            raise ValueError
        else:
            sensor_path[starting_sensor][time] = to_edge
    else:
        print("CONSTRUCTION ERROR: sensor_path was not initalized for {}".format(starting_sensor))
        inspect_data_structures(current_sensor, time, to_edge, sensor_path)
        raise ValueError
    return sensor_path

def edge_already_exists(current_sensor, time, edge_list):
    """
    see if there aready is an edge @ time in edge_list
    """
    if current_sensor in edge_list.keys():
        if time in edge_list[current_sensor].keys():
            return True
    return False

In [52]:
def get_adjacent_vertex(current_sensor, edge_list, t):
    """
    get the vertex connect to the current sensor at time t
    """
    if(current_sensor in edge_list.keys()):
        if(t in edge_list[current_sensor].keys()):
            return edge_list[current_sensor][t]
    return None

def is_tower(current_sensor, tower_names=TOWERS.keys()):
    """
    return true of the sensor is a tower
    """
    if(current_sensor in tower_names):
        return True
    else:
        return False

In [79]:
def validate_path(starting_sensor, sensor_path, edge_list):
    """
    1. sensor_path[starting_sensor] ends at a tower
    2. all edges from sensor_path are in edge list
    """
    # VALIDATE CONSTRUCTION FOR THE STARTING VERTEX
    time = 0
    current_sensor = starting_sensor
    next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    while(next_vertex):
        current_sensor = next_vertex
        time = time + 1
        next_vertex = get_adjacent_vertex(current_sensor, edge_list, time)
    try:
        assert is_tower(current_sensor)
    except AssertionError:
        inspect_data_structures(starting_sensor, time, next_vertex, edge_list)
        inspect_data_structures(starting_sensor, time, next_vertex, sensor_path)
        raise AssertionError

In [101]:
# {vertex : {0:next0, 1:next1, 2:next2}}
sensor_path = {}
# {vertex: {0: next0}, 
#  next0: {1: next1},
#  next1: {2: next2}
edge_list = {}
TIME_BOUND = 5

# for each sensor in SENSORS
for i, (starting_sensor, val) in enumerate(SENSORS.items()):
    # initailize variables
    tower_path = False
    time = 0
    visited_on_path = [starting_sensor]
    sensor_path[starting_sensor] = {}
    current_sensor = starting_sensor
    while(not(tower_path) and (time < TIME_BOUND)):
        # give it an edge and neighbor at time 0
        if edge_already_exists(current_sensor, time, edge_list):
            # setting this to break the while loop
            tower_path = True
        else:
            random.shuffle(sensor_neighbors[current_sensor])
            to_edge = sensor_neighbors[current_sensor][0][0]
            if to_edge in visited_on_path:
                # avoiding cycles
                # we want the while loop to fail if this happens
                tower_path = True
            else:
                visited_on_path.append(to_edge)
                edge_list = update_edge_list(current_sensor, time, to_edge, edge_list)
                sensor_path = update_sensor_path(starting_sensor, time, to_edge, sensor_path)
                time = time + 1
                current_sensor = to_edge
    if(time == TIME_BOUND):
        # did we already add a tower at this time? 
        if(edge_already_exists(current_sensor, time, edge_list)):
            # assert an adjacent tower
            assert is_tower(get_adjacent_vertex(current_sensor, edge_list, time)) 
        else:
            ## add an edge from the current sensor to a tower
            random.shuffle(sensor_tower_neighbors[current_sensor])
            to_tower = sensor_tower_neighbors[current_sensor][0][0]
            edge_list = update_edge_list(current_sensor, time, to_tower, edge_list)
            sensor_path = update_sensor_path(starting_sensor, time, to_tower, sensor_path)
    else:
        ## add an edge from the current sensor to a tower
        if(not(edge_already_exists(current_sensor, time, edge_list))):
            random.shuffle(sensor_tower_neighbors[current_sensor])
            to_tower = sensor_tower_neighbors[current_sensor][0][0]
            edge_list = update_edge_list(current_sensor, time, to_tower, edge_list)
            sensor_path = update_sensor_path(starting_sensor, time, to_tower, sensor_path)
    validate_path(starting_sensor, sensor_path, edge_list)

In [83]:
time

0

In [97]:
not(tower_path) and (time < TIME_BOUND)

False

In [72]:
tower_path and (time < TIME_BOUND)

False

In [90]:
time = 4
tower_path = True
print(time)
print(tower_path)
not(tower_path) and (time < TIME_BOUND)

4
True


False

In [95]:
edge_already_exists("126951211", 1, edge_list)

True

In [104]:
path = ["a", "b", "a"]

In [105]:
assert len(path) == len(set(path))

AssertionError: 

In [106]:
str(path)

"['a', 'b', 'a']"

In [111]:
test = {"a":[(0,1),(1,1),(2,1)]}

In [112]:
test

{'a': [(0, 1), (1, 1), (2, 1)]}

In [113]:
neighbors = [item[0] for item in test["a"]]

In [114]:
neighbors

[0, 1, 2]

In [115]:
neighbors[0:len(neighbors)-1]

[0, 1]

In [116]:
neighbors[len(neighbors)-1:]

[2]

In [118]:
test = [[0,1],[2,3]]
for [a,b] in test:
    print(a)
    print(b)

0
1
2
3


In [123]:
ex = test[-1]
print(ex)

[2, 3]


In [125]:
for a, b in *ex:
    print(a)
    print(b)

SyntaxError: invalid syntax (<ipython-input-125-851c55bc2128>, line 1)